In [1]:
email_conversation = """From: 테디 (teddy@teddynote.com)
To: 이은채 대리님 (eunchae@teddyinternational.me)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, 이은채 대리님,

저는 테디노트의 테디입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가진 기업으로, 귀사의 요구에 맞는 최적의 솔루션을 제공할 수 있다고 자부합니다.

저희 테디노트의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보입니다. 이 솔루션은 특히 다양한 산업에서의 성공적인 적용 사례를 통해 그 효과를 입증하였습니다.

귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 만나 뵐 수 있을까요?

미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. 이은채 대리님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

테디
테디노트 AI 솔루션팀"""

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")
    company: str = Field(description="메일을 보낸 사람의 회사정보")

In [3]:
# LCEL구조

#chain  = prompt | llm | output_parser

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOpenAI(model_name="gpt-4o-mini")
output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [5]:
from langchain_core.prompts import PromptTemplate

template = """
You are a helpful assistant. Please answer the following questions in KOREAN.

#Question:
다음의 이메일 내용 중에서 주요 내용을 추출해 주세요.

#Email Converation:
{email_conversation}

#Format:
{format}
"""

prompt = PromptTemplate.from_template(template)

prompt = prompt.partial(format=output_parser.get_format_instructions())

In [6]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"title": "Person", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\uba54\uc77c \uc81c\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ud14d\uc2a4\ud2b8", "type": "string"}, "date": {"title": "Date", "description"

In [7]:
chain = prompt | llm | output_parser

In [8]:
# 객체로 만들어준다. 체인실행.
answer = chain.invoke({"email_conversation": email_conversation})

In [9]:
answer

EmailSummary(person='테디', email='teddy@teddynote.com', subject='RAG 솔루션 시연 관련 미팅 제안', summary='테디노트의 RAG 솔루션을 소개하고, 귀사와의 협력을 논의하기 위한 미팅을 제안합니다.', date='2023-07-18T10:00:00', company='테디노트')

In [10]:
answer.summary

'테디노트의 RAG 솔루션을 소개하고, 귀사와의 협력을 논의하기 위한 미팅을 제안합니다.'

In [11]:
#answer.phone_number

In [12]:
answer.company

'테디노트'

# 검색: SERP API
https://serpapi.com/integrations/python

- prompt에 추가정보로 search한것을 넣어준다.

In [13]:
# import os
# os.environ["SERPAPI_API_KEY"] = "9020cdf9235a65b087f324a802b283ff67ac56cf8ddf0ab5bc8522c284d4a573"

In [14]:
from langchain_community.utilities import SerpAPIWrapper
# hl은 google ui language, gl은 country
params = {
    "engine" : "google",
    "gl" : "kr",
    "hl" : "ko",
    "num" : "3"
}

search = SerpAPIWrapper(params=params)

In [15]:
search.run("테디노트 site:github.com")

"['✔️ 코드스테이츠 X 테디노트 - 깃헙 블로그 제작하기 강의. ✔️ SK그룹 - 2023년 상반기 텐서플로우 딥러닝 과정 강의. ✔️ S-Oil - 파이썬 데이터 분석, 머신러닝 과정 강의 ...', '차분하게 배워볼 수 있는 유튜브 채널. 테디노트. 텐서플로우 관련 영상들이 주를 이룹니다. 데이터 분석, 머신러닝, 그리고 딥러닝 주제를 다루는 유튜브 채널.']"

In [16]:
search.run("테디노트 site:naver.com")

'["... 테디노트 (teddylee777.github.io)운영(\'17년~현재) 전 - 삼성전자SCSA1기 - 삼성전자무선사업부삼성노트,테마,갤러리 앱개발 - C-LAB사내벤처독립 ...", \'이런 고민을 해결해 주고 손쉽게 만들 수 있는 방법을 찾아 보았다. 그 시작의 최고는 테디노트님께서 만든 10분만에 블로그 만들기 였다. 나의 부족한 ...\']'

In [17]:
search.run("테디노트")

'[\'데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \\u200d♂️ ...more ...more fastcampus.co.kr/data_online_teddyand 2 ...\', \'LangGraph Retrieval Agent는 언어 처리, AI 모델 통합, 데이터베이스 관리, 그래프 기반 데이터 처리 등 다양한 기능을 제공하여 언어 기반 AI 애플리케이션 개발에 ...\', \'블로그와 유튜브 "테디노트"를 운영하고 있으며, "파이썬 딥러닝 텐서플로"를 집필하였습니다. 데이터분석과 AI를 사랑하고 지식공유에 활발히 참여하고 있습니다.\']'

In [18]:
# Pydantic이 들어있는 prompt의 chain. 내용은 email_conversation
answer.person

'테디'

In [19]:
query = f"{answer.person} {answer.company} {answer.email}"
query

'테디 테디노트 teddy@teddynote.com'

In [20]:
search_result = search.run(query)

In [21]:
search_result

'[\'테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...\', \'테디노트 TeddyNote. @teddynote. 25.2K subscribers•188 videos. 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \\u200d ...\', \'테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 ... TeddyNote. Enter to select. to navigate. ESC to close. 테디노트.dev. Teddy Lee (이경록) ...\']'

In [22]:
type(search_result)

str

In [23]:
# 리스트 형태로 저장. 이메일 정보 + 보낸사람의 보낸 정보를 바탕으로 또하나의 체인을 통하여 최종 정리를 한다.
search_result = eval(search_result)

In [24]:
type(search_result)

list

In [25]:
search_result

['테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...',
 '테디노트 TeddyNote. @teddynote. 25.2K subscribers•188 videos. 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d ...',
 '테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 ... TeddyNote. Enter to select. to navigate. ESC to close. 테디노트.dev. Teddy Lee (이경록) ...']

In [26]:
# 위 리스트를 하나로 병합해준다. 검색 결과를 스트링으로 만들어준다
search_result_string = '\n'.join(search_result)

In [27]:
search_result_string

'테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...\n테디노트 TeddyNote. @teddynote. 25.2K subscribers•188 videos. 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \u200d ...\n테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 ... TeddyNote. Enter to select. to navigate. ESC to close. 테디노트.dev. Teddy Lee (이경록) ...'

In [28]:
answer

EmailSummary(person='테디', email='teddy@teddynote.com', subject='RAG 솔루션 시연 관련 미팅 제안', summary='테디노트의 RAG 솔루션을 소개하고, 귀사와의 협력을 논의하기 위한 미팅을 제안합니다.', date='2023-07-18T10:00:00', company='테디노트')

In [29]:
### report_chain 검색된 결과를 가지고 정리.

In [30]:
from langchain_core.prompts import PromptTemplate

template = """
당신은 이메일의 주요 정보를 바탕으로 요약 정리해 주는 전문가 입니다.
당신의 의무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
주어진 정보를 기반으로 양식(format)에 맞추어 요약을 작성해 주세요.
답변에는 카테고리 별로 emoji를 적극 활용하여 답변해 주세요.


# Inforamtion:
- Sender: {sender}
- Additional Information about sender: {additional_information}
- Company: {company}
- Email: {email}
- Subject: {subject}
- Summay: {summary}
- Date: {date}

# Format(in markdown format):
😀보낸 사람:
- (보낸 사람의 이름, 회사 정보):

📧이메일 주소:
- (보낸 사람의 이메일 주소)

😍보낸 사람과 관련하여 검색된 추가 정보:
- (검색된 추가 정보)

❗주요 내용:
- (이메일, 제목, 요약)

⏰일정:
- (미팅 날짜 및 시간)

# Answer:
"""

report_prompt = PromptTemplate.from_template(
    template
)

In [31]:
from langchain_core.output_parsers import StrOutputParser

In [32]:
report_chain = report_prompt | llm | StrOutputParser()

In [33]:
report_response = report_chain.invoke({
    "sender": answer.person,
    "additional_information": search_result_string,
    "company": answer.company,
    "email": answer.email,
    "subject": answer.subject,
    "summary": answer.summary,
    "date": answer.date
})

In [34]:
print(report_response)

😀보낸 사람:
- 테디, 테디노트

📧이메일 주소:
- teddy@teddynote.com

😍보낸 사람과 관련하여 검색된 추가 정보:
- 테디노트는 데이터 분석, 머신러닝, 딥러닝, LLM에 대한 내용을 다루는 채널로, 연구보다는 개발에 중점을 둔 개발자입니다. 테디노트는 25.2K 구독자를 보유하고 있으며, 188개의 동영상을 업로드했습니다. 

❗주요 내용:
- 이메일 제목: RAG 솔루션 시연 관련 미팅 제안
- 요약: 테디노트의 RAG 솔루션을 소개하고, 귀사와의 협력을 논의하기 위한 미팅을 제안합니다.

⏰일정:
- 미팅 날짜 및 시간: 2023-07-18 10:00 AM


In [35]:
chain = prompt | llm | output_parser